**Date :** Created on Thursday January 7 2021

**Group 8 - Innovation**

**abstract_notebook_v0** 

**@author :** Flora Estermann, Katia Belaid, Dirat Tanguy.  

**Description :** This notebook breaks down into several stages. The first will first lemmatize our articles then will apply a preproccesing to all of our articles. The second, then, will create summaries according to a fixed number of sentences or words.

# Part 1 : Install / Download / Import Librairy

## Install librairy

In [ ]:
#! python -m spacy download fr_core_news_md
#! pip3 install 'https://github.com/explosion/spacy-models/releases/download/fr_core_news_md-2.3.0/fr_core_news_md-2.3.0.tar.gz'

## Download Librairy

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Boulanger\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Boulanger\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Import librairy

### - Usefull librairy :

In [2]:
import pandas as pd
import numpy as np
import spacy
from tqdm import trange

### - Text librairy :

In [3]:
import fr_core_news_md
import re
import string
import unicodedata
from nltk import word_tokenize
from nltk.corpus import stopwords

### - Machine Learning Librairy :

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Part 2 : Data Loading

In [5]:
def Load_data(helper_path : str) -> pd.DataFrame:
    """Documentation
    
    Parameters :
        - helper_path : the file path

    Output (if exists) :
        - df : My Dataframe cleaned and reindexed

    """
    
    # Data Load with pandas librairy
    df = pd.read_json(helper_path)

    # Drop articles with no content
    df = df[df['art_content'] != '']

    # Reset my dataframe index
    df = df.reset_index(drop = True)
    
    # Returns my clean dataframe
    return df

In [6]:
# My file path for the fonction
Helper_path : str = 'C:/Users/Boulanger/Documents/interpromo_2021/df_deduplicated_v4.json'

# My DataFrame variable
My_data : pd.DataFrame = Load_data(Helper_path)

# To show my DataFrame
My_data.head(10)

,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,1,La FNCDG et l’ANDCDG ont publié en septembre l...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,None,None
1,2,Malgré la levée des mesures de confinement le ...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,None,None
2,25,Quels étaient les objectifs poursuivis par le ...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,fr,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2019/10/in...,None,None
3,27,"La journée thématique, qui aura lieu durant le...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,fr,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/pu...,None,None
4,28,La 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,fr,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/03/Sa...,None,None
5,30,L’une des innovations de la loi n°2019-828 du ...,"<p style=""text-align: justify;"">L’une des inno...",22 octobre 2020,fr,La publication d’un guide d’accompagnement à l...,http://fncdg.com/la-publication-dun-guide-dacc...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/10/LG...,None,None
6,31,"La FNCDG mène, en collaboration avec d’autres ...","<p style=""text-align: justify;"">La FNCDG mène,...",10 décembre 2020,fr,La publication d’un guide de sensibilisation a...,http://fncdg.com/la-publication-dun-guide-de-s...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/12/im...,None,None
7,32,"Créé pour et par les décideurs territoriaux, É...","<p style=""text-align: justify;"">Créé pour et p...",24 février 2017,fr,Lancement du réseau Étoile,http://fncdg.com/lancement-du-reseau-etoile/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/02/re...,None,None
8,34,Les décrets n°2017-397 et n°2017-318 du 24 mar...,"<p style=""text-align: justify;"">Les décrets n°...",5 avril 2017,fr,Le cadre d’emplois des agents de police munici...,http://fncdg.com/le-cadre-demplois-des-agents-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/04/po...,None,None
9,35,Une candidate à un examen professionnel organi...,"<p style=""text-align: justify;"">Une candidate ...",6 juillet 2017,fr,Le Conseil d’Etat confirme la souveraineté des...,http://fncdg.com/le-conseil-detat-confirme-la-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/07/Co...,None,None


# Part 3 : Data Preprocessing

## Method 1 : Lemmatization Data

In [7]:
# Load the french model for the lemmatization
Nlp = spacy.load('fr_core_news_md')

In [8]:
def Lemmatize(sentence: str) -> str :
    """Documentation
    
    Parameters :
        - sentence: all sentences not lemmatized 
            in the column ["art_content"]

    Output (if exists) :
        - sentence: all sentences lemmatize. 
            thanks to the library 'fr_core_web_sm'  

    """
    
    # Use my lemmatization model
    s = Nlp(sentence)
    
    # My initialized sentence
    lemmatized : str = ''
    
    # Step 1 : Lemmatization of each word
    for w in s:
        
        # Lemmatization of my sentence
        lemmatized += w.lemma_ + ' '
    
    # Return my lemmatized sentence
    return lemmatized

In [10]:
# Lemmatization of each sentences
for i in trange(len(My_data) , position = 0 , leave = True):
    
    My_data.loc[i, 'art_content'] = Lemmatize(My_data.loc[i, 'art_content'])

# To show my DataFrame Lemmatized
My_data.head()

100%|██████████████████████████████████████████████████████████████████████████████| 7490/7490 [14:18<00:00,  8.72it/s]


,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,1,le FNCDG et l’ andcdg avoir publier en septemb...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,None,None
1,2,malgré le levée un mesure de confinement le 11...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,None,None
2,25,quel être le objectif poursuivre par le gouver...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,fr,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2019/10/in...,None,None
3,27,"le journée thématique , qui avoir lieu durant ...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,fr,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/pu...,None,None
4,28,le 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,fr,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/03/Sa...,None,None


## Method 2 : Data Cleanning

In [11]:
def Cleanning(item: str, special_character : list) -> str :
    """
    Parameters :
        - item: all articles without the removal of unnecessary words
        (for example : "stopWords") in the column ["art_content"]
        - special_character: a list of specials characters to 
        remove to the articles in the column ["art_content"]

    Output (if exists) :
        - result: all articles without unnecessary words 
        and characters 
        
    """

    # Convert text to lowercase
    item : str = item.lower()

    # Remove mail
    item : str = re.sub("(\w+)@(\w+).(\w+)","",item)

    # Remove twitter name
    item : str = re.sub("@(\w+)","",item)

    # Remove site ".com"
    item : str = re.sub("(\S+).com(\S+)","",item)
    item : str = re.sub("(\S+).com","",item)
        
    # Remove site ".fr"   
    item : str = re.sub("(\S+).fr(\S+)","",item)
    item : str = re.sub("(\S+).fr","",item)

    # Remove numbers
    # item : str = re.sub(r'\d+', '', item)

    # Remove hastags
    item : str = re.sub("#(\w+)","",item)

    # Remove years
    # item : str = re.sub("en (\d+)","",item)
    
    # Remove punctuation
    item : str = item.translate(str.maketrans("", "", string.punctuation))
    
    # Step 1 : Remove French accents
    for i in range(len(item)):
    
        # Get the article
        try:
        
            # Transform to 'utf-8'
            item = unicode(item , 'utf-8')
    
        except NameError: # unicode is a default on python 3 
        
            pass
        
        # Remove the accents
        item = unicodedata.normalize('NFD', str(item)) \
                            .encode('ascii', 'ignore') \
                            .decode("utf-8")
    
    # Step 2 : Remove Special character 
    for i in special_character:
        
        item = item.replace(i, "")
    
    # Remove stopwords
    stop_words = set(nltk.corpus.stopwords.words('french'))
    tokens = word_tokenize(item)
    result = [i for i in tokens if not i in stop_words]
    result = " ".join(result)
    
    # Remove whitespaces
    result = result.strip()
    
    # Return my cleaned article
    return result

In [12]:
# My initialized column
My_data['art_clean'] : pd.DataFrame = ''

# Create my list of special characters
Special_character : list = ["!","\"","#","$","%","&","\\","(", \
                            ")","*","+",",","-",".","/",":",";", \
                            "<","=",">","?","@","[","]","^","_", \
                            "{","|","}","~","«","»","’","•","…", \
                            "â","€","™","—","�","–","“","”"]

# Cleanning of each articles
for i in trange(len(My_data) , position = 0 , leave = True):
    
    My_data.loc[i, 'art_clean'] = Cleanning(My_data.loc[i, 'art_content'], \
                                            Special_character)

# To show my DataFrame Cleanned
My_data.head()

100%|██████████████████████████████████████████████████████████████████████████████| 7490/7490 [26:06<00:00,  4.78it/s]


,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag,art_clean
0,1,le FNCDG et l’ andcdg avoir publier en septemb...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,None,None,fncdg andcdg avoir publier septembre 9em editi...
1,2,malgré le levée un mesure de confinement le 11...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,None,None,malgre levee mesure confinement 11 mai 2020 pl...
2,25,quel être le objectif poursuivre par le gouver...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,fr,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2019/10/in...,None,None,quel etre objectif poursuivre gouvernement cad...
3,27,"le journée thématique , qui avoir lieu durant ...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,fr,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/pu...,None,None,journee thematique avoir lieu durant salon pre...
4,28,le 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,fr,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/03/Sa...,None,None,1ere journee thematique region theme vers nouv...


# Part 4 : Creation of text summaries

## Method 1 : considering the number of sentences

In [13]:
def Summary_sentences(item : str, long_res_max : int) -> list :
    """Documentation
    
    Parameters :
        - item: all summary press articles, after the preprocessing, 
            in the column ["art_content"]
        - long_res_max : Maximum length of our summary

    Output (if exists) :
        - text_summary: My text summary with max length condition
            (n sentences with the best score). 

    """
    
    # My initialized text summary
    text_summary : list = []
    
    # Copy of each article
    sentences : str = item.copy()
    
    # Step 1 : Build Summary with conditions
    while len(text_summary) < long_res_max \
        and len(text_summary) < len(sentences):
    
        # TF-IDF initialization
        vectorizer = TfidfVectorizer()
        
        # TF-IDF of each sentences
        X = vectorizer.fit_transform(sentences)

        # Sentence weight
        s_weight = X.sum(axis = 1) # sum of columns
        
        # List of sentence weight
        s_weight = s_weight.tolist()
      
        # Add sentence with the best score
        text_summary.append(sentences[np.argmax(s_weight)])
        
        # Updated sentences List
        sentences.remove(sentences[np.argmax(s_weight)])
    
    # Return my text summary
    return text_summary

In [16]:
# My initialized column
My_data['abstract_sentence'] : pd.DataFrame = ""
    
# Text summary of each article
for i in trange(len(My_data) , position = 0 , leave = True):
  
    # Split text into sentences
    Tok_art = nltk.tokenize.sent_tokenize(My_data.loc[i, 'art_clean'])
  
    # Get the abstract
    try:
        
        # Summary creation
        Res : list = Summary_sentences(Tok_art, 5)
    
    # If the abstract is None
    except ValueError: 
        
        Res : str = ''
    
    # Add result to dataframe
    My_data.loc[i, 'abstract_sentence'] = ' '.join(Res)

# To show my summary
My_data.head()

100%|█████████████████████████████████████████████████████████████████████████████| 7490/7490 [00:18<00:00, 394.52it/s]


,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag,art_clean,abstract_sentence
0,1,le FNCDG et l’ andcdg avoir publier en septemb...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,None,None,fncdg andcdg avoir publier septembre 9em editi...,fncdg andcdg avoir publier septembre 9em editi...
1,2,malgré le levée un mesure de confinement le 11...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,None,None,malgre levee mesure confinement 11 mai 2020 pl...,malgre levee mesure confinement 11 mai 2020 pl...
2,25,quel être le objectif poursuivre par le gouver...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,fr,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2019/10/in...,None,None,quel etre objectif poursuivre gouvernement cad...,quel etre objectif poursuivre gouvernement cad...
3,27,"le journée thématique , qui avoir lieu durant ...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,fr,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/pu...,None,None,journee thematique avoir lieu durant salon pre...,journee thematique avoir lieu durant salon pre...
4,28,le 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,fr,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/03/Sa...,None,None,1ere journee thematique region theme vers nouv...,1ere journee thematique region theme vers nouv...


## Method 2 : considering the number of words

In [17]:
def Summary_words(item : str, nb_words : int) -> list :
    """Documentation
    
    Parameters :
        - item: all summary press articles, after the preprocessing, 
            in the column ["art_content"]
        - nb_words : Maximum number words of our summary

    Output (if exists) :
        - text_summary: My text summary with the first n_words of the article.

    """
    
    # Takes the first n words of the article.
    text_summary = " ".join(item.split()[:nb_words])

    return text_summary

In [19]:
# My initialized column
My_data['abstract_words'] : pd.DataFrame = ""
    
# Text summary of each article
for i in trange(len(My_data) , position = 0 , leave = True):
    
    My_data.loc[i, 'abstract_words'] = Summary_words(My_data.loc[i, 'art_clean'], 50)

# To show my summary
My_data.head()

100%|████████████████████████████████████████████████████████████████████████████| 7490/7490 [00:04<00:00, 1648.22it/s]


,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag,art_clean,abstract_sentence,abstract_words
0,1,le FNCDG et l’ andcdg avoir publier en septemb...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,None,None,fncdg andcdg avoir publier septembre 9em editi...,fncdg andcdg avoir publier septembre 9em editi...,fncdg andcdg avoir publier septembre 9em editi...
1,2,malgré le levée un mesure de confinement le 11...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,None,None,malgre levee mesure confinement 11 mai 2020 pl...,malgre levee mesure confinement 11 mai 2020 pl...,malgre levee mesure confinement 11 mai 2020 pl...
2,25,quel être le objectif poursuivre par le gouver...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,fr,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2019/10/in...,None,None,quel etre objectif poursuivre gouvernement cad...,quel etre objectif poursuivre gouvernement cad...,quel etre objectif poursuivre gouvernement cad...
3,27,"le journée thématique , qui avoir lieu durant ...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,fr,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/pu...,None,None,journee thematique avoir lieu durant salon pre...,journee thematique avoir lieu durant salon pre...,journee thematique avoir lieu durant salon pre...
4,28,le 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,fr,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/03/Sa...,None,None,1ere journee thematique region theme vers nouv...,1ere journee thematique region theme vers nouv...,1ere journee thematique region theme vers nouv...


# Part 5 : Export data

In [22]:
# The .csv export data
My_data.to_csv('abstract_v1.csv', index = False)